In [3]:
!pip install queuelib

In [8]:
import requests as rq
from bs4 import BeautifulSoup as bs
from collections import OrderedDict as od
from threading import Thread
from queuelib import queue
from datetime import datetime
import time
import random
import json
import csv
import traceback as tb
import re

In [7]:
#設定8條執行序
NUM_THREADS = 8
#6秒超時
timeoutSec = 6

In [9]:
HOST = 'https://www.famicloud.com.tw'

In [10]:
index_url = HOST

# 上層爬蟲

In [196]:
def famicloud_crawler():
    res_index = rq.get(index_url)
    soup_index = bs(res_index.text, 'lxml')
    soup_li = soup_index.li
    
    
    for i in range (1 ,1000):
        try:
            small_branch = soup_li.select('a')[i].text
            pre_small_branch_url = soup_li.select('a')[i]['href']
            small_branch_url = "{}{}".format(index_url, pre_small_branch_url)
            
            print(small_branch_url)
            #交給中層爬蟲
            pass

        except KeyError:       
            sub_branch = soup_li.select('a')[i].text
            #print(sub_branch)

        except IndexError:
            break   

    print("[INFO] famicloud_crawler complete!")

In [197]:
famicloud_crawler()

https://www.famicloud.com.tw/Branchs/Branch/Index/1/563/564
https://www.famicloud.com.tw/Branchs/Branch/Index/1/287/288
https://www.famicloud.com.tw/Branchs/Branch/Index/1/908/914
https://www.famicloud.com.tw/Branchs/Branch/Index/1/565/566
https://www.famicloud.com.tw/Branchs/Branch/Index/312/313/316
https://www.famicloud.com.tw/Branchs/Branch/Index/312/895/896
https://www.famicloud.com.tw/Branchs/Branch/Index/312/895/897
https://www.famicloud.com.tw/Branchs/Branch/Index/312/356/357
https://www.famicloud.com.tw/Branchs/Branch/Index/312/356/358
https://www.famicloud.com.tw/Branchs/Branch/Index/312/548/549
https://www.famicloud.com.tw/Branchs/Branch/Index/312/607/608
https://www.famicloud.com.tw/Branchs/Branch/Index/312/504/505
https://www.famicloud.com.tw/Branchs/Branch/Index/312/504/506
https://www.famicloud.com.tw/Branchs/Branch/Index/312/504/507
https://www.famicloud.com.tw/Branchs/Branch/Index/312/504/508
https://www.famicloud.com.tw/Branchs/Branch/Index/2/9/27
https://www.famicloud

# 下層爬蟲

In [198]:
def product_page_crawler(product_url):
    res = rq.get(product_url)
    soup = bs(res.text, 'lxml')
    
    product_dict = od()
    food_info_dict = od()  
    
    food_name = ''
    
    try:
        soup.select("div")[55].strong.text
        content = soup.select("div")[55]
        new_content = content
        
        while(new_content.strong != None):
            
            try:
                food_name += new_content.strong.text
                new_content.strong.extract()
                
            except AttributeError:
                break
                
        food_info_dict["food_name"] = food_name
        print(food_info_dict)
        
    except AttributeError:
        soup.select("div")[54].strong.text
        content = soup.select("div")[54]
        new_content = content
        
        while(new_content.strong != None):
            
            try:   
                food_name += new_content.strong.text
                new_content.strong.extract()
                
            except AttributeError:
                break
                
        food_info_dict["food_name"] = food_name
        
    except:
        print("[***ERROR***]can not find food_name!")
        
    while(True):
        
        try:
            new_content.br.replace_with(' ')
            
        except AttributeError:
            break
   
    food_info_and_other = new_content.text.strip().split() 
    
    #original_price
    food_info_dict["original_price"] = soup.select(".BOriginalPrice")[0].text
    #discount_price
    food_info_dict["discount_price"] = soup.select(".OnSalePrice > b")[0].text
    #stored_method
    food_info_dict["stored_method"] = soup.select(".BProductLabel")[0].img['alt']
    #category
    category_list = []
    a_hrefs = soup.select('#map-list')[0]
    branch_levels = a_hrefs.select('a')
    for branch in branch_levels:
        if(branch.text != '首頁'):
            category_list.append(branch.text)
    food_info_dict["category"] = category_list
    #weight
    food_info_dict["weight"] = food_info_and_other[1]
    #expired_date
    food_info_dict["expired_date"] = food_info_and_other[7]
    #country_of_origin
    food_info_dict["country_of_origin"] = food_info_and_other[5]
    
    #把food_info字典放入product_dict字典
    product_dict["food_info"] = food_info_dict
    
    #steps(未完成)
    #product_dict["steps"] = null
    
    #product_img_url
    product_dict["product_img_url"] = soup.select(".product_img_box")[0].img['src']
    #product_characteristic_img_url
    
    try:
        #不在P標籤內
        product_dict["product_characteristic_img_url"] = soup.select('div > img')[1]['src']
        #rint(product_dict)
        
    except IndexError:
        #在P標籤內
        product_dict["product_characteristic_img_url"] = soup.select("div > p > img")[0]['src']
    
    #web_url
    product_dict["web_url"] = product_url
    
    print(product_dict)

# 單頁測試

In [199]:
product_url = 'https://www.famicloud.com.tw/Branchs/Branch/Product/3/13/50/3872'

In [200]:
product_page_crawler(product_url)

OrderedDict([('food_info', OrderedDict([('food_name', '家香豬五花火鍋肉片'), ('original_price', '原價$168'), ('discount_price', '88'), ('stored_method', '冷凍'), ('category', ['肉品肉類', '豬肉', '肉片\\肉條']), ('weight', '300g±5%/包'), ('expired_date', '180天'), ('country_of_origin', '台灣')])), ('product_img_url', 'https://familycloud.blob.core.windows.net/products/10001653_0.jpg'), ('product_characteristic_img_url', 'https://familycloud.blob.core.windows.net/products/3872/1653-2NEW2.jpg'), ('web_url', 'https://www.famicloud.com.tw/Branchs/Branch/Product/3/13/50/3872')])


In [212]:
product_page_crawler('https://www.famicloud.com.tw/Branchs/Branch/Product/2/10/42/6012')

AttributeError: 'NoneType' object has no attribute 'text'

In [224]:
res = rq.get('https://www.famicloud.com.tw/Branchs/Branch/Product/2/10/42/6012')
soup = bs(res.text, 'lxml')

In [235]:
for div_index in range(70,1, -1):
    if hasattr(soup.select("div")[div_index].strong, 'text'):
        print(div_index)

52
48
35
26


In [233]:
soup.select("div")[52]

<div>
<strong>鮮之流。北太平洋半殼鮑魚</strong><br/>重量/容量:360g±20g<br/>內容物:鮑魚<br/>產地:中國<br/>有效期限:240天<br/>食用方式:退冰後，沖一下擦乾，可煎、蒸或烤。<br/>保存方式: 冷凍<br/>食品業登錄字號: A-124549062-00000-0<br/><br/>◎本產品造型、顏色以實物為主。 <br/>◎商品圖片僅供實物參考。內容物組成以實物及商品說明為主。 <br/>◎注意事項：本為食品特殊類別，ㄧ經拆封或食物、包裝碰撞變形或保存不良導致變質、非運送過程失溫導致食品變質者，恕無法退換貨，敬請見諒與配合。 <br/>◎退貨事項：除商品本身有瑕疵可辦理退貨，商品一經使用或損毀即不可退貨，退貨必須保留紙箱及商品。
                </div>

# 多頁測試

In [210]:
product_url_list = [
    'https://www.famicloud.com.tw/Branchs/Branch/Product/3/13/50/3872',
    #https://www.famicloud.com.tw/Branchs/Branch/Product/312/504/505/1170',
    #https://www.famicloud.com.tw/Branchs/Branch/Product/3/14/59/1288',
    'https://www.famicloud.com.tw/Branchs/Branch/Product/2/10/42/6012',
    'https://www.famicloud.com.tw/Branchs/Branch/Product/2/12/49/4258',
    'https://www.famicloud.com.tw/Branchs/Branch/Product/622/745/758/4118',
    'https://www.famicloud.com.tw/Branchs/Branch/Product/622/1278/1279/6289',
    'https://www.famicloud.com.tw/Branchs/Branch/Product/5/233/903/5056',
    'https://www.famicloud.com.tw/Branchs/Branch/Product/4/1055/1056/5286',
    'https://www.famicloud.com.tw/Branchs/Branch/Product/5/1023/1070/7001',
    'https://www.famicloud.com.tw/Branchs/Branch/Product/254/344/614/6688',
    'https://www.famicloud.com.tw/Branchs/Branch/Product/879/1016/1017/6361',
    'https://www.famicloud.com.tw/Branchs/Branch/Product/948/967/968/6951'
]

In [211]:
for product_url in product_url_list:
    product_page_crawler(product_url)

OrderedDict([('food_info', OrderedDict([('food_name', '家香豬五花火鍋肉片'), ('original_price', '原價$168'), ('discount_price', '88'), ('stored_method', '冷凍'), ('category', ['肉品肉類', '豬肉', '肉片\\肉條']), ('weight', '300g±5%/包'), ('expired_date', '180天'), ('country_of_origin', '台灣')])), ('product_img_url', 'https://familycloud.blob.core.windows.net/products/10001653_0.jpg'), ('product_characteristic_img_url', 'https://familycloud.blob.core.windows.net/products/3872/1653-2NEW2.jpg'), ('web_url', 'https://www.famicloud.com.tw/Branchs/Branch/Product/3/13/50/3872')])


AttributeError: 'NoneType' object has no attribute 'text'